In [10]:
# Data
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import impl.utils as utils

# Dataset preparation and pre-processing
mnist = input_data.read_data_sets('data/MNIST_data/', one_hot=False)
X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels
# y_test.shape, y_val.shape, y_train.shape
M, D, C = X_train.shape[0], X_train.shape[1], y_train.max() + 1
# M, D, C
X_train, X_val, X_test = utils.prepro(X_train, X_val, X_test)
# X_train.shape, X_val.shape, X_test.shape
# if net_type == 'cnn':
img_shape = (1, 28, 28)
X_train = X_train.reshape(-1, *img_shape)
X_val = X_val.reshape(-1, *img_shape)
X_test = X_test.reshape(-1, *img_shape)
X_train.shape, X_val.shape, X_test.shape

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


((55000, 1, 28, 28), (5000, 1, 28, 28), (10000, 1, 28, 28))

In [11]:
# Model
# import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.NN as nn

class DCNN(nn.NN):

    def __init__(self, D, C, H, p_dropout, lam=1e-3, loss='cross_ent', nonlin='relu'):
        self.p_dropout = p_dropout # prob, keep_prob, or prob_dropout
        self.loss = loss # minimum cross_entropy
        self.mode = 'classification'
        super().__init__(D, C, H, lam, p_dropout, loss, nonlin)
        
    def _init_model(self, D, C, H):
        self.model = []
        self.model.append(dict(
            W1=np.random.randn(H, 1, 3, 3) / np.sqrt(H / 2.),
            b1=np.zeros((H, 1))
        ))
        self.model.append(dict(
            W2=np.random.randn(H, H, 3, 3) / np.sqrt(H / 2.),
            b2=np.zeros((H, 1)),
            gamma2=np.ones((1, H*D)),
            beta2=np.zeros((1, H*D)),

            W22=np.random.randn(H, H, 3, 3) / np.sqrt(H / 2.),
            b22=np.zeros((H, 1)),
            gamma22=np.ones((1, H*D)),
            beta22=np.zeros((1, H*D)),

            W23=np.random.randn(H, H, 3, 3) / np.sqrt(H / 2.),
            b23=np.zeros((H, 1)),
            gamma23=np.ones((1, H*D)),
            beta23=np.zeros((1, H*D)),

            W24=np.random.randn(H, H, 3, 3) / np.sqrt(H / 2.),
            b24=np.zeros((H, 1)),
            gamma24=np.ones((1, H*D)),
            beta24=np.zeros((1, H*D)),

            W25=np.random.randn(H, H, 3, 3) / np.sqrt(H / 2.),
            b25=np.zeros((H, 1)),
            gamma25=np.ones((1, H*D)),
            beta25=np.zeros((1, H*D)),
        ))
        self.model.append(dict(
            W3=np.random.randn(H*D, C) / np.sqrt(H*D / 2.),
            b3=np.zeros((1, C))
        ))
        self.bn_caches = dict(            
            bn2_mean=np.zeros((1, H*D)),
            bn2_var=np.zeros((1, H*D)),

            bn22_mean=np.zeros((1, H*D)),
            bn22_var=np.zeros((1, H*D)),

            bn23_mean=np.zeros((1, H*D)),
            bn23_var=np.zeros((1, H*D)),

            bn24_mean=np.zeros((1, H*D)),
            bn24_var=np.zeros((1, H*D)),

            bn25_mean=np.zeros((1, H*D)),
            bn25_var=np.zeros((1, H*D))
        )
        
    def forward(self, X, train=False):

        # 1st layer: Input to Conv
        h1, h1_cache = l.conv_forward(X=X, W=self.model[0]['W1'], b=self.model[0]['b1']) 
        h1, nl1_cache = l.relu_forward(h1)

        ############################################################################################
        # midst layer: Convnet 1
        h2, h2_cache = l.conv_forward(X=h1, W=self.model[1]['W2'], b=self.model[1]['b2'])
        h2 = h2.reshape([nl1_cache.shape[0], -1])
        h2, bn2_cache, self.bn_caches['bn2_mean'], self.bn_caches['bn2_var'] = l.bn_forward(h2, 
                                                self.model[1]['gamma2'], self.model[1]['beta2'], 
                                                (self.bn_caches['bn2_mean'], self.bn_caches['bn2_var']), 
                                                train=train)
        h2, nl2_cache = l.relu_forward(h2)
        do2_cache = None # ERROR: referenced before assigned!
        if train: h2, do2_cache = l.dropout_forward(X=h2, p_dropout=self.p_dropout)
        
        # midst layer: Convnet 2
        h2 = h2.reshape(nl1_cache.shape)
        h2, h22_cache = l.conv_forward(X=h2, W=self.model[1]['W22'], b=self.model[1]['b22'])
        h2 = h2.reshape([nl1_cache.shape[0], -1])
        h2, bn22_cache, self.bn_caches['bn22_mean'], self.bn_caches['bn22_var'] = l.bn_forward(h2, 
                                                self.model[1]['gamma22'], self.model[1]['beta22'], 
                                                (self.bn_caches['bn22_mean'], self.bn_caches['bn22_var']), 
                                                train=train)
        h2, nl22_cache = l.relu_forward(h2)
        do22_cache = None # ERROR: referenced before assigned!
        if train: h2, do22_cache = l.dropout_forward(X=h2, p_dropout=self.p_dropout)

        # midst layer: Convnet 3
        h2 = h2.reshape(nl1_cache.shape)
        h2, h23_cache = l.conv_forward(X=h2, W=self.model[1]['W23'], b=self.model[1]['b23'])
        h2 = h2.reshape([nl1_cache.shape[0], -1])
        h2, bn23_cache, self.bn_caches['bn23_mean'], self.bn_caches['bn23_var'] = l.bn_forward(h2, 
                                                self.model[1]['gamma23'], self.model[1]['beta23'], 
                                                (self.bn_caches['bn23_mean'], self.bn_caches['bn23_var']), 
                                                train=train)
        h2, nl23_cache = l.relu_forward(h2)
        do23_cache = None # ERROR: referenced before assigned!
        if train: h2, do23_cache = l.dropout_forward(X=h2, p_dropout=self.p_dropout)
            
        # midst layer: Convnet 4
        h2 = h2.reshape(nl1_cache.shape)
        h2, h24_cache = l.conv_forward(X=h2, W=self.model[1]['W24'], b=self.model[1]['b24'])
        h2 = h2.reshape([nl1_cache.shape[0], -1])
        h2, bn24_cache, self.bn_caches['bn24_mean'], self.bn_caches['bn24_var'] = l.bn_forward(h2, 
                                                self.model[1]['gamma24'], self.model[1]['beta24'], 
                                                (self.bn_caches['bn24_mean'], self.bn_caches['bn24_var']), 
                                                train=train)
        h2, nl24_cache = l.relu_forward(h2)
        do24_cache = None # ERROR: referenced before assigned!
        if train: h2, do24_cache = l.dropout_forward(X=h2, p_dropout=self.p_dropout)
            
        # midst layer: Convnet 5
        h2 = h2.reshape(nl1_cache.shape)
        h2, h25_cache = l.conv_forward(X=h2, W=self.model[1]['W25'], b=self.model[1]['b25'])
        h2 = h2.reshape([nl1_cache.shape[0], -1])
        h2, bn25_cache, self.bn_caches['bn25_mean'], self.bn_caches['bn25_var'] = l.bn_forward(h2, 
                                                self.model[1]['gamma25'], self.model[1]['beta25'], 
                                                (self.bn_caches['bn25_mean'], self.bn_caches['bn25_var']), 
                                                train=train)
        h2, nl25_cache = l.relu_forward(h2)
        do25_cache = None # ERROR: referenced before assigned!
        if train: h2, do25_cache = l.dropout_forward(X=h2, p_dropout=self.p_dropout)
        ############################################################################################
            
        # last layer : FC to Output
        h3, h3_cache = l.fc_forward(X=h2, W=self.model[2]['W3'], b=self.model[2]['b3'])

        cache = (h1_cache, nl1_cache, h2_cache, bn2_cache, nl2_cache, do2_cache, h22_cache, bn22_cache, nl22_cache, do22_cache, h23_cache, bn23_cache, nl23_cache, do23_cache, h24_cache, bn24_cache, nl24_cache, do24_cache, h25_cache, bn25_cache, nl25_cache, do25_cache, h3_cache)
        return h3, cache
    
    def loss_function(self, y, y_train):
#         loss = self.loss_funs[self.loss](self.model[3], y, y_train, self.lam)
        loss = self.loss_funs[self.loss](y, y_train)
        dy = self.dloss_funs[self.loss](y, y_train)
        return loss, dy
    
    def backward(self, dy, cache):

        h1_cache, nl1_cache, h2_cache, bn2_cache, nl2_cache, do2_cache, h22_cache, bn22_cache, nl22_cache, do22_cache, h23_cache, bn23_cache, nl23_cache, do23_cache, h24_cache, bn24_cache, nl24_cache, do24_cache, h25_cache, bn25_cache, nl25_cache, do25_cache, h3_cache = cache

        # last layer
        dh2, dw3, db3 = l.fc_backward(dout=dy, cache=h3_cache)
        
        # midst layer 5
        dh2 = l.dropout_backward(dout=dh2, cache=do25_cache)
        dh2 = l.relu_backward(dout=dh2, cache=nl25_cache)
        dh2, dgamma25, dbeta25 = l.bn_backward(dout=dh2, cache=bn25_cache)
        dh2 = dh2.reshape(nl1_cache.shape)
        dh2, dw25, db25 = l.conv_backward(dout=dh2, cache=h25_cache)
        dh2 = dh2.reshape([nl1_cache.shape[0], -1])

        # midst layer 4
        dh2 = l.dropout_backward(dout=dh2, cache=do24_cache)
        dh2 = l.relu_backward(dout=dh2, cache=nl24_cache)
        dh2, dgamma24, dbeta24 = l.bn_backward(dout=dh2, cache=bn24_cache)
        dh2 = dh2.reshape(nl1_cache.shape)
        dh2, dw24, db24 = l.conv_backward(dout=dh2, cache=h24_cache)
        dh2 = dh2.reshape([nl1_cache.shape[0], -1])
        
        # midst layer 3
        dh2 = l.dropout_backward(dout=dh2, cache=do23_cache)
        dh2 = l.relu_backward(dout=dh2, cache=nl23_cache)
        dh2, dgamma23, dbeta23 = l.bn_backward(dout=dh2, cache=bn23_cache)
        dh2 = dh2.reshape(nl1_cache.shape)
        dh2, dw23, db23 = l.conv_backward(dout=dh2, cache=h23_cache)
        dh2 = dh2.reshape([nl1_cache.shape[0], -1])
        
        # midst layer 2
        dh2 = l.dropout_backward(dout=dh2, cache=do22_cache)
        dh2 = l.relu_backward(dout=dh2, cache=nl22_cache)
        dh2, dgamma22, dbeta22 = l.bn_backward(dout=dh2, cache=bn22_cache)
        dh2 = dh2.reshape(nl1_cache.shape)
        dh2, dw22, db22 = l.conv_backward(dout=dh2, cache=h22_cache)
        dh2 = dh2.reshape([nl1_cache.shape[0], -1])

        # midst layer 1
        dh2 = l.dropout_backward(dout=dh2, cache=do2_cache)
        dh2 = l.relu_backward(dout=dh2, cache=nl2_cache)
        dh2, dgamma2, dbeta2 = l.bn_backward(dout=dh2, cache=bn2_cache)
        dh2 = dh2.reshape(nl1_cache.shape)
        dh1, dw2, db2 = l.conv_backward(dout=dh2, cache=h2_cache)

        # 1st layer
        dh1 = l.relu_backward(dout=dh1, cache=nl1_cache)
        dX, dw1, db1 = l.conv_backward(dout=dh1, cache=h1_cache)

        # grad for GD
        grad = []
        grad.append(dict(
            W1=dw1, 
            b1=db1
        ))
        grad.append(dict(
            W2=dw2, 
            b2=db2,
            gamma2=dgamma2,
            beta2=dbeta2,
            
            W22=dw22, 
            b22=db22,
            gamma22=dgamma22,
            beta22=dbeta22,

            W23=dw23, 
            b23=db23,
            gamma23=dgamma23,
            beta23=dbeta23,
            
            W24=dw24, 
            b24=db24,
            gamma24=dgamma24,
            beta24=dbeta24,

            W25=dw25, 
            b25=db25,
            gamma25=dgamma25,
            beta25=dbeta25
        ))
        grad.append(dict(
            W3=dw3, 
            b3=db3
        ))
        
        return dX, grad
    
    def test(self, X):
        y_logit, cache = self.forward(X, train=False)
        y_prob = util.softmax(y_logit)
        if self.mode == 'classification':
            return np.argmax(y_prob, axis=1)
        else: # self.mode == 'regression'
            return np.round(y_logit)

In [12]:
# SGD
# import numpy as np
import impl.utils as util
import impl.constant as c
import copy
from sklearn.utils import shuffle as skshuffle


def get_minibatch(X, y, minibatch_size, shuffle=True):
    minibatches = []

    if shuffle:
        X, y = skshuffle(X, y)

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]

        minibatches.append((X_mini, y_mini))

    return minibatches

def adam(nn, X_train, y_train, val_set=None, alpha=0.001, mb_size=256, n_iter=2000, print_after=100):
    M, R = [], []
    M.append({k: np.zeros_like(v) for k, v in nn.model[0].items()})
    R.append({k: np.zeros_like(v) for k, v in nn.model[0].items()})
    M.append({k: np.zeros_like(v) for k, v in nn.model[1].items()})
    R.append({k: np.zeros_like(v) for k, v in nn.model[1].items()})
    M.append({k: np.zeros_like(v) for k, v in nn.model[2].items()})
    R.append({k: np.zeros_like(v) for k, v in nn.model[2].items()})
    beta1 = .9
    beta2 = .999

    minibatches = get_minibatch(X_train, y_train, mb_size)

    if val_set:
        X_val, y_val = val_set

    for iter in range(1, n_iter + 1):
        t = iter
        idx = np.random.randint(0, len(minibatches))
        X_mini, y_mini = minibatches[idx]

        #         grad, loss = nn.train_step(X_mini, y_mini)
        #         def train_step(self, X_train, y_train):
        #         """
        #         Single training step over minibatch: forward, loss, backprop
        #         """
        y, cache = nn.forward(X_mini, train=True)
        loss, dy = nn.loss_function(y, y_mini)
        dX, grad = nn.backward(dy, cache)
        #         return grad, loss

        if iter % print_after == 0:
            if val_set:
                val_acc = util.accuracy(y_val, nn.test(X_val))
                print('Iter-{} loss: {:.4f} validation: {:4f}'.format(iter, loss, val_acc))
            else:
                print('Iter-{} loss: {:.4f}'.format(iter, loss))

        for k in grad[0]:
            M[0][k] = util.exp_running_avg(M[0][k], grad[0][k], beta1)
            R[0][k] = util.exp_running_avg(R[0][k], grad[0][k]**2, beta2)

            m_k_hat = M[0][k] / (1. - beta1**(t))
            r_k_hat = R[0][k] / (1. - beta2**(t))

            nn.model[0][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)
            
        for k in grad[1]:
            M[1][k] = util.exp_running_avg(M[1][k], grad[1][k], beta1)
            R[1][k] = util.exp_running_avg(R[1][k], grad[1][k]**2, beta2)

            m_k_hat = M[1][k] / (1. - beta1**(t))
            r_k_hat = R[1][k] / (1. - beta2**(t))

            nn.model[1][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)
            
        for k in grad[2]:
            M[2][k] = util.exp_running_avg(M[2][k], grad[2][k], beta1)
            R[2][k] = util.exp_running_avg(R[2][k], grad[2][k]**2, beta2)

            m_k_hat = M[2][k] / (1. - beta1**(t))
            r_k_hat = R[2][k] / (1. - beta2**(t))

            nn.model[2][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

    return nn

In [13]:
# Hyper-parameters
n_iter = 100 # number of epochs
alpha = 1e-3 # learning_rate
mb_size = 64 # width, timestep for sequential data or minibatch size
num_layers = 1 # depth 
print_after = 1 # print loss for train, valid, and test

In [ ]:
net = DCNN(C=C, D=D, H=8, p_dropout=0.5)

net = adam(nn=net, X_train=X_train, y_train=y_train, val_set=(X_val, y_val), mb_size=mb_size, alpha=alpha, 
           n_iter=n_iter, print_after=print_after)

y_pred = net.predict(X_test)
accs = np.mean(y_pred == y_test)

print()
print('Test Mean accuracy: {:.4f}, std: {:.4f}'.format(accs.mean(), accs.std()))

Iter-1 loss: 3.1479 validation: 0.106600
Iter-2 loss: 3.0901 validation: 0.095800
Iter-3 loss: 3.0528 validation: 0.096000
Iter-4 loss: 3.1227 validation: 0.096600
Iter-5 loss: 2.6991 validation: 0.096800
Iter-6 loss: 2.7405 validation: 0.095800
Iter-7 loss: 2.8990 validation: 0.095800
Iter-8 loss: 2.5033 validation: 0.098600
Iter-9 loss: 2.4098 validation: 0.098600
Iter-10 loss: 2.2245 validation: 0.098600
Iter-11 loss: 2.6369 validation: 0.098600
Iter-12 loss: 2.1226 validation: 0.098600
Iter-13 loss: 1.7940 validation: 0.098600
Iter-14 loss: 1.9219 validation: 0.098600
Iter-15 loss: 2.3919 validation: 0.098600
Iter-16 loss: 2.0263 validation: 0.098600
Iter-17 loss: 1.9438 validation: 0.098600
Iter-18 loss: 2.1798 validation: 0.110000
Iter-19 loss: 2.3725 validation: 0.110000
Iter-20 loss: 1.7419 validation: 0.109800
Iter-21 loss: 1.7917 validation: 0.109800
Iter-22 loss: 1.8384 validation: 0.111200
Iter-23 loss: 2.0731 validation: 0.111600
Iter-24 loss: 1.9383 validation: 0.110200
I